1. 원본 기사에 NER
- (가능하면) 제목 + 본문 전체로 NER
- 추출된 엔터티가 종목명 사전에 있으면 → 매칭 완료

2. 요약 후 텍스트에 NER
- 본문이 길거나, 종목 언급이 흩어져 있으면 NER이 놓칠 수 있음
- 간단 요약(문장 2~3개) 후 다시 NER
- 요약 모델은 Kanghoon/KoBART-summarization (뉴스요약 SOTA) or digit82/kobart-summarizer 등
- 요약본에서 종목명이 새로 잡히면 → 매칭

3. 둘 다 실패(종목명 없음) → BERT 분류
- 종목명 추출이 안된 기사만 따로 뽑아서
- 멀티라벨 BERT/DeBERTa 분류모델로 (예: snunlp/KR-FinBERT or kaist-ai/KF-Deberta-v1-base)
- 임베딩 유사도 기반 매칭 or 직접 멀티라벨 파인튜닝 모델

# 1. 기사 본문에서 NER

In [11]:
# 데이터, 종목명 사전 불러오기
import pandas as pd

news_df = pd.read_csv('/Users/JooAnLee/final_project/db/news_2023_2025.csv')
kospi_df = pd.read_csv('/Users/JooAnLee/final_project/db/KRX_KOSPI.csv', encoding='cp949')

In [12]:
news_df.info()
print('-----------------------------------------------------')
kospi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58405 entries, 0 to 58404
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   news_id  58405 non-null  object
 1   wdate    58405 non-null  object
 2   title    58405 non-null  object
 3   article  58405 non-null  object
 4   press    58405 non-null  object
 5   url      58405 non-null  object
 6   image    58405 non-null  object
dtypes: object(7)
memory usage: 3.1+ MB
-----------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 845 entries, 0 to 844
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   종목코드    845 non-null    int64  
 1   종목명     845 non-null    object 
 2   종가      845 non-null    int64  
 3   대비      845 non-null    int64  
 4   등락률     845 non-null    float64
 5   상장시가총액  845 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 39.7+ KB


In [7]:
from transformers import pipeline
from tqdm import tqdm

# 모델명
model_name = "mepi/KR-FinBert-finetuned-ner"

# 파이프라인 생성
ner_pipe = pipeline(
    task="ner",
    model=model_name,
    tokenizer=model_name,
    aggregation_strategy="simple"  # 엔터티 병합 옵션 (예: "ORG", "PER" 등 전체 단어 단위로 묶어줌)
)

/opt/anaconda3/envs/final_jn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


In [8]:
# 종목명만 set으로 (사전에 NaN 제거)
company_names = set(kospi_df['종목명'].dropna().unique())

In [13]:
# 제목 + 본문 결합
news_df['text_combined'] = news_df['title'].fillna('') + ' ' + news_df['article'].fillna('')
news_df['text_combined'] = news_df['text_combined'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [22]:
# KLUE NER 라벨 매핑 (정확한 라벨)
id2label = {
    0: 'B-DT', 1: 'I-DT',
    2: 'B-LC', 3: 'I-LC',
    4: 'B-OG', 5: 'I-OG',  # 조직명 라벨
    6: 'B-PS', 7: 'I-PS',
    8: 'B-QT', 9: 'I-QT',
    10: 'B-TI', 11: 'I-TI',
    12: 'O'
}

In [23]:
# 텍스트 분할 함수
def split_text(text, max_len):
    return [text[i:i+max_len] for i in range(0, len(text), max_len)]

# 수정된 종목명 추출 함수
def extract_stock_entities_with_label(text, id2label, max_length=600):
    chunks = split_text(text, max_length)
    matched_names = set()
    
    for chunk in chunks:
        entities = ner_pipe(chunk)
        merged_entities = []
        current_word = ""
        current_label = ""
        
        for ent in entities:
            word = ent["word"]
            entity_group = ent["entity_group"]
            
            # 'LABEL_4' → 4로 변환
            label_num = int(entity_group.split("_")[1])
            label_name = id2label[label_num]
            
            # B-xxx/I-xxx 병합
            if label_name.startswith("B-"):
                if current_word:
                    merged_entities.append((current_word, current_label))
                current_word = word
                current_label = label_name[2:]  # B-OG → OG
            elif label_name.startswith("I-"):
                current_word += word.replace("##", "")
            else:
                if current_word:
                    merged_entities.append((current_word, current_label))
                    current_word = ""
                    current_label = ""
        
        if current_word:
            merged_entities.append((current_word, current_label))
        
        # OG 라벨만 추출 (조직명)
        for word, label in merged_entities:
            if label == "OG":  # 수정된 부분
                if word in company_names:
                    matched_names.add(word)
    
    return list(matched_names)

In [24]:
# 500개 샘플 추출
sample_df = news_df.sample(500, random_state=42).copy()

# 종목명 추출 실행
tqdm.pandas()
sample_df["ner_stocks"] = sample_df["text_combined"].progress_apply(
    lambda x: extract_stock_entities_with_label(x, id2label)
)

# 결과 확인
print("종목명이 추출된 기사 수:", sample_df[sample_df["ner_stocks"].str.len() > 0].shape[0])
print("추출된 종목명 예시:")
for idx, stocks in sample_df[sample_df["ner_stocks"].str.len() > 0]["ner_stocks"].head().items():
    print(f"기사 {idx}: {stocks}")

100%|██████████| 500/500 [01:09<00:00,  7.22it/s]

종목명이 추출된 기사 수: 178
추출된 종목명 예시:
기사 24743: ['삼성화재', '한화']
기사 12522: ['LG유플러스']
기사 44072: ['롯데칠성']
기사 8581: ['고려아연']
기사 26610: ['셀트리온']


In [25]:
# 종목명이 추출된 기사 수와 비율 계산
num_articles_with_stocks = sample_df[sample_df['ner_stocks'].map(len) > 0].shape[0]
total_articles = sample_df.shape[0]
ratio = (num_articles_with_stocks / total_articles) * 100

print(f"전체 기사 수: {total_articles}개")
print(f"종목명이 추출된 기사 수: {num_articles_with_stocks}개")
print(f"추출 비율: {ratio:.1f}%")

# 추가로 추출된 종목명 통계도 확인
total_stocks_extracted = sum(len(stocks) for stocks in sample_df['ner_stocks'])
print(f"총 추출된 종목명 개수: {total_stocks_extracted}개")

# 가장 많이 언급된 종목 확인
from collections import Counter
all_stocks = [stock for stocks in sample_df['ner_stocks'] for stock in stocks]
stock_counts = Counter(all_stocks)
print(f"\n가장 많이 언급된 종목 TOP 5:")
for stock, count in stock_counts.most_common(5):
    print(f"  {stock}: {count}회")

전체 기사 수: 500개
종목명이 추출된 기사 수: 178개
추출 비율: 35.6%
총 추출된 종목명 개수: 240개

가장 많이 언급된 종목 TOP 5:
  셀트리온: 13회
  한화: 9회
  SK: 9회
  고려아연: 6회
  현대로템: 6회


# 2, 코스피 텍스트 매칭 후 NER

In [28]:
# 2단계: 텍스트매칭
sample_df["matched_stocks"] = sample_df["text_combined"].apply(lambda x: keyword_match_stocks(x, company_names))

# 3단계: 매칭 통계 계산
matched_count = (sample_df["matched_stocks"].apply(lambda x: len(x) > 0)).sum()
total_count = len(sample_df)
matched_percent = matched_count / total_count * 100

print(f"종목명 1개 이상 매칭된 기사 수: {matched_count}/{total_count} ({matched_percent:.1f}%)")

종목명 1개 이상 매칭된 기사 수: 398/500 (79.6%)


In [30]:
print("매칭된 종목명 예시 (기사 제목, 본문 일부 포함):")
for idx, row in sample_df[sample_df["matched_stocks"].str.len() > 0].head().iterrows():
    print(f"\n[기사 {idx}]")
    print("제목:", row['title'])
    print("본문(일부):", row['article'][:100], "...")
    print("매칭된 종목명:", row['matched_stocks'])

매칭된 종목명 예시 (기사 제목, 본문 일부 포함):

[기사 24743]
제목: “올 3분기 국내 오피스 4조원 거래…낮은 공실률 당분간 지속”
본문(일부): 프라임 오피스 거래 규모
올 3분기 국내 오피스 투자시장의 거래가 활발하게 진행된 것으로 나타났다.
23일 글로벌 부동산 솔루션 업체인 세빌스코리아에 따르면 올 3분기 오피스 투자 ...
매칭된 종목명: ['한화리츠', '삼성생명', '한화', 'TP', '삼성화재']

[기사 34686]
제목: 비투엔, IDC건립 위한 화성 일대 토지 매입…"신성장 동력 마련"
본문(일부): 빅데이터·인공지능(AI) 전문기업 비투엔이 신성장동력 확보에 본격 시동을 건다.
비투엔은 관계사 아이오케이컴퍼니와 손잡고 특수목적법인(SPC) 에이아이링크를 통해 경기도 화성 일대 ...
매칭된 종목명: ['한화']

[기사 12522]
제목: LG유플러스, 올해 전략적 변화 시작…실적 회복 전망-NH
본문(일부): [이데일리 이용성 기자] NH투자증권은 7일 LG유플러스(032640)에 대해 지난해 실적이 부진했으나 올해 전략적 변화가 시작되며 실적 회복도 이뤄질 것이라고 밝혔다. 투자의견은 ...
매칭된 종목명: ['LG', 'NH투자증권', 'LG헬로비전', 'LG유플러스']

[기사 44072]
제목: 하이투자증권 "롯데칠성, 비수기 끝 성수기 기대"
본문(일부): [롯데칠성음료 유튜브 캡처. 재판매 및 DB 금지]
(서울=연합뉴스) 조성흠 기자 = 하이투자증권은 3일 롯데칠성에 대해 올해 1분기 실적이 시장 기대치에 못 미쳤으나 2, 3분기 ...
매칭된 종목명: ['롯데칠성', 'DB']

[기사 35322]
제목: K팝에 이어 K푸드, 10년 사이 60% 성장
본문(일부): [파이낸셜뉴스] 한국 경제의 위상이 높아지고 K-콘텐츠 파급력이 커지면서 K-푸드(Food) 시장이 급성장하고 있다. K-푸드 인기 품목이 만두, 라면, 김밥 등으로 확장되고, 국 ...
매칭된 종목명: ['선진', '대상']
